#### Import libraries

In [1]:
import json
import math
import os
import shutil
from tkinter import *
from tkinter import ttk
from tkinter import filedialog
from tkinter import font

##### Initialize Interface

In [2]:
filename = 'information.json'
root = Tk()
frm = ttk.Frame(root, padding=10)
frm.grid()
app_width = 500
app_height = 500
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
x = (screen_width / 2) - (app_width / 2)
y = (screen_height / 2) - (app_height / 2)
root.geometry(f'{app_width}x{app_height}+{int(x)}+{int(y)}')

''

##### Initialize files

In [3]:
def getDir(library_path):
    if library_path and os.path.isdir(library_path):
        # Loop through each folder within the library path directory
        items = os.listdir(library_path)
        # Separate folders and files
        folders = [item for item in items if os.path.isdir(os.path.join(library_path, item))]
        
        # Save folders to a file
        with open('avaliable_anime.txt', 'w') as file:
            for folder in folders:
                file.write(f"{folder}\n")
    else:
        print("ERROR: no directory listed!!! Add it to the information.json file manually...")

In [4]:
if not os.path.isfile(filename):
        # If it doesn't exist, create it and write data to it
        data = {
            "PC Directory": "testing",
            "PC Capacity": "testing",
            "PC storage usage": "testing",
            "Phone storage usage": "testing",
            "Phone Directory": "testing",
            "Phone Capacity": "testing"
        }
        with open(filename, 'w') as json_file:
            json.dump(data, json_file, indent=4)

# Read data from json file
with open(filename, 'r') as json_file:
    data = json.load(json_file)

# There is a directory that the user has entered
if os.path.isdir(data["PC Directory"]):
    getDir(data["PC Directory"])


# TODO
# The user has not updated the directory to read from
#else:
    # Gray out the button for transfering anime.
    # Gray out the PC capacity
    # Gray out the limit for pc limit
    # gray out anime to transfer
    # Gray out the get list of avaliable anime button
    # Gray out the transfer button.
    # Pretty much wait until the user has chosen a valid directory

#### Functions

##### Percentage of file transfered

In [5]:
def file_transfer_progress(src, dst, buffer_size=1024 * 1024):  # 1MB buffer
    total_size = os.path.getsize(src)
    transferred = 0

    with open(src, "rb") as fsrc, open(dst, "wb") as fdst:
        while chunk := fsrc.read(buffer_size):
            fdst.write(chunk)
            transferred += len(chunk)
            yield transferred / total_size  # Return progress as a float (0.0 - 1.0)

##### Get size of directory

In [6]:
def get_directory_size(directory):
    total_size = 0
    # Walk through each folder, subfolder, and files
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            # Get the full file path
            file_path = os.path.join(dirpath, filename)
            # Add the file size to the total size
            if os.path.isfile(file_path):
                total_size += os.path.getsize(file_path)
                
    if total_size == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB")
    i = int(math.floor(math.log(total_size, 1024)))
    p = math.pow(1024, i)
    s = round(total_size / p, 2)
    return f"{s} {size_name[i]}"

##### Get avaliable storage within a directory

In [7]:
def get_available_storage(directory="."):
    total, used, free = shutil.disk_usage(directory)
    return free

##### Progress bar

In [8]:
def senseStorage(src_dir, dst_dir, root2):
    # Sense how much the size of the files is to be transfered
    
    total = float(str.split(get_directory_size(src_dir))[0])
    # Instantiate the progress bar
    my_progress = ttk.Progressbar(root2, orient=HORIZONTAL, length=270, mode='determinate')
    my_progress.grid()
    
    progress = 0
    size_name = ("B", "KB", "MB", "GB", "TB")
    i = int(math.floor(math.log(total, 1024)))
    p = math.pow(1024, i)
    s = round(total / p, 2)
    print(f"{s} {size_name[i]}")
    # Periodically check the size of the files within the src_dir and update the progress variable
    shutil.move(src_dir, dst_dir)
    while progress != total:
        # Get the progress
        progress = total - int(str.strip(get_directory_size(src_dir))[0])
        print(f"updated progress: {progress}")
        perc_prog = (progress / total) * 100
        print(f"percent progress: {perc_prog}")
        # Update the progress bar variable
        my_progress['value'] = perc_prog

##### Retrieve Anime

In [9]:
def retrieve(titles, data, root2):
    # For each title in the list of shows
    for title in titles:
        
        # Get directory of show folder
        src_dir = os.path.join(data["PC Directory"], title).replace("/", "\\")
        
        # Destination folder
        dest_dir = os.path.join(data["Phone Directory"], title).replace("/", "\\")
        
        # Phone directory
        phone_dir = data["Phone Directory"]
        
        # Check if the show folder exists
        if not os.path.isdir(src_dir):
            print(f"ERROR: Source folder '{src_dir}' does not exist!")
            continue
        
        # Check if the destination folder already exists
        if not os.path.isdir(dest_dir) and os.path.isdir(phone_dir):
            
            senseStorage(src_dir, data["Phone Directory"], root2)
            
            print(f"Retrieved '{title}' to '{phone_dir}'.")
        else:
            print(f"ERROR: Destination path '{dest_dir}' already exists!!!")

##### Store Anime

In [10]:
def store(titles, data, root2):
    
    # For each title in the list of shows
    for title in titles:
        
        # Get directory of show folder
        src_dir = os.path.join(data["Phone Directory"], title).replace("/", "\\")
        
        # Destination folder
        dest_dir = os.path.join(data["PC Directory"], title).replace("/", "\\")
        
        # Phone directory
        pc_dir = data["PC Directory"]
        
        # Check if the show folder exists
        if not os.path.isdir(src_dir):
            print(f"ERROR: Source folder '{src_dir}' does not exist!")
            continue
        
        # Check if the destination folder already exists
        if not os.path.isdir(dest_dir) and os.path.isdir(pc_dir):
                        
            for progress in file_transfer_progress(src_dir, pc_dir):
                print(f"Progress (new stuff): {progress:.2%}")  # Prints progress as percentage'''
                
            # Move folder and its contents to the new folder
            senseStorage(src_dir, pc_dir, root2)
            
            print(f"Transferred '{title}' to '{pc_dir}'.")
        else:
            print(f"ERROR: Destination path '{dest_dir}' already exists!!!")
            
            
# TODO
    # Add in a tracker for the files being transfered.


#### Buttons

##### Set PC Directory

In [11]:
def setPCDir():
    if not os.path.isfile(filename):
        # If it doesn't exist, create it and write data to it
        data = {
            "PC Directory": "testing",
            "PC Capacity": "testing",
            "PC storage usage": "testing",
            "Phone storage usage": "testing",
            "Phone Directory": "testing",
            "Phone Capacity": "testing"
        }
        with open(filename, 'w') as json_file:
            json.dump(data, json_file, indent=4)
    folder_path = filedialog.askdirectory()
    if folder_path:
        ttk.Label(frm, text=folder_path).grid(column=1, row=1)
        # Read the existing JSON data to update it
        with open(filename, 'r') as json_file:
            data = json.load(json_file)
        
        # Update the relevant fields in the data dictionary
        data["PC Directory"] = folder_path
        
        # Write the updated data back to the JSON file
        with open(filename, 'w') as json_file:
            json.dump(data, json_file, indent=4)
        print(f"{filename} has been updated with the new PC Directory.")

##### Set Phone Directory

In [12]:
def setPhoneDir():
    if not os.path.isfile(filename):
        # If it doesn't exist, create it and write data to it
        data = {
            "PC Directory": "testing",
            "PC Capacity": "testing",
            "PC storage usage": "testing",
            "Phone storage usage": "testing",
            "Phone Directory": "testing",
            "Phone Capacity": "testing"
        }
        with open(filename, 'w') as json_file:
            json.dump(data, json_file, indent=4)
    folder_path = filedialog.askdirectory()
    if folder_path:
        ttk.Label(frm, text=folder_path).grid(column=1, row=2)
        # Read the existing JSON data to update it
        with open(filename, 'r') as json_file:
            data = json.load(json_file)
        
        # Update the relevant fields in the data dictionary
        data["Phone Directory"] = folder_path
        
        # Write the updated data back to the JSON file
        with open(filename, 'w') as json_file:
            json.dump(data, json_file, indent=4)
        print(f"{filename} has been updated with the new Phone Directory.")

##### Transfer Anime

In [13]:
def transferAnime():
    # Run a new instance of ttk
    root2 = Tk()
    frm2 = ttk.Frame(root2, padding=10)
    frm2.grid()
    app_width = 500
    app_height = 500
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    x = (screen_width / 2) - (app_width / 2)
    y = (screen_height / 2) - (app_height / 2)
    root2.geometry(f'{app_width}x{app_height}+{int(x + 100)}+{int(y + 100)}')
    
    # Get a list of the anime inside of the folder
    with open(filename, 'r') as json_file:
        data = json.load(json_file)
    if data:
        # Get the directory of the library
        library_path = data["PC Directory"]

        # Loop through each folder within the library path directory
        items = os.listdir(library_path)
        # Separate folders and files
        folders = [item for item in items if os.path.isdir(os.path.join(library_path, item))]

        # Save folders to a file
        with open('avaliable_anime.txt', 'w') as file:
            for folder in folders:
                file.write(f"{folder}\n")
    
    # Instantiate and grid scrollbar and listbox
    my_scrollbar = Scrollbar(frm2, orient=VERTICAL)
    my_listbox = Listbox(frm2, width=50, yscrollcommand=my_scrollbar.set, selectmode=MULTIPLE)
    my_scrollbar.config(command=my_listbox.yview)
    frm2.grid()
    my_listbox.grid(pady=15)
    my_listbox.grid(row=0, column=0)
    my_scrollbar.grid(row=0, column=1, sticky='ns')
    
    # Add shows to listbox from avaliable anime
    shows = []
    with open('avaliable_anime.txt', 'r') as file:
        for line in file:
            shows.append(line)
    for show in shows:
        my_listbox.insert(0, show)
    
    
    # Create an entry box
    my_entry = Entry(root2)
    my_entry.grid()
    
    # Update entrybox with listbox clicked
    def fillout(e):
        selection = my_listbox.curselection()
        
        # Delete whatever is in the entrybox
        my_entry.delete(0, END)
        
        # Add clicked list item to entry box (should I sub this out with anchor?????)
        my_entry.insert(0, my_listbox.get(selection[-1]))
        
    # Update search bar with entry from listbox
    my_listbox.bind("<<ListboxSelect>>", fillout)
    
    
    def update(data):
        
        my_listbox.delete(0, END)
        
        # Add toppings to listbox
        for show in data:
            my_listbox.insert(END, show)
    
    
    
    # Checks entry vs listbox
    def check(e):
        
        # Grab what was typed
        typed = my_entry.get()
        
        if typed == '':
            data = shows
        else:
            data = []
            for show in shows:
                if typed.lower() in show.lower():
                    data.append(show)
                    
        # Update our listbox with selected items
        update(data)
        # See if the string is within the listbox item list
    my_entry.bind("<KeyRelease>", check)
    
    # Select All Button to show entries as labels
    def transferSelected():
        test = []
        for item in my_listbox.curselection():
            test.append(str.strip((my_listbox.get(item)))       )
        combined_text = ', '.join(test)
        combined_label = Label(root2, text=combined_text)
        combined_label.grid()  # Place it below the individual labels
        # Function to transfer the items selected
        retrieve(test, data, root2)
        for item in reversed(my_listbox.curselection()):
            my_listbox.delete(item)

    # Transfer Selected titles
    transferSelected_button = Button(root2, text="Retrieve Selected", command=transferSelected)
    transferSelected_button.grid(pady=10)
    
    
    my_label = Label(root2, text='')
    my_label.grid(pady=5)

##### Store Anime

In [ ]:
def storeAnime():
    # Run a new instance of ttk
    root2 = Tk()
    frm2 = ttk.Frame(root2, padding=10)
    frm2.grid()
    app_width = 500
    app_height = 500
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    x = (screen_width / 2) - (app_width / 2)
    y = (screen_height / 2) - (app_height / 2)
    root2.geometry(f'{app_width}x{app_height}+{int(x + 100)}+{int(y + 100)}')
    # Get a list of the anime inside of the folder
    with open(filename, 'r') as json_file:
        data = json.load(json_file)
    if data:
        # Get the directory of the library
        library_path = data["Phone Directory"]

        # Loop through each folder within the library path directory
        items = os.listdir(library_path)
        # Separate folders and files
        folders = [item for item in items if os.path.isdir(os.path.join(library_path, item))]

        # Save folders to a file
        with open('phone_anime.txt', 'w') as file:
            for folder in folders:
                file.write(f"{folder}\n")
    
    # Instantiate and grid scrollbar and listbox
    my_scrollbar = Scrollbar(frm2, orient=VERTICAL)
    my_listbox = Listbox(frm2, width=50, yscrollcommand=my_scrollbar.set, selectmode=MULTIPLE)
    my_scrollbar.config(command=my_listbox.yview)
    frm2.grid()
    my_listbox.grid(pady=15)
    my_listbox.grid(row=0, column=0)
    my_scrollbar.grid(row=0, column=1, sticky='ns')
    
    # Add shows to listbox from avaliable anime
    shows = []
    with open('phone_anime.txt', 'r') as file:
        for line in file:
            shows.append(line)
    for show in shows:
        my_listbox.insert(0, show)
    
    
    # Create an entry box
    my_entry = Entry(root2)
    my_entry.grid()
    
    # Update entrybox with listbox clicked
    def fillout(e):
        selection = my_listbox.curselection()
        
        # Delete whatever is in the entrybox
        my_entry.delete(0, END)
        
        # Add clicked list item to entry box (should I sub this out with anchor?????)
        my_entry.insert(0, my_listbox.get(selection[-1]))
        
    # Update search bar with entry from listbox
    my_listbox.bind("<<ListboxSelect>>", fillout)
    
    
    def update(data):
        
        my_listbox.delete(0, END)
        
        # Add toppings to listbox
        for show in data:
            my_listbox.insert(END, show)
    
    # Checks entry vs listbox
    def check(e):
        
        # Grab what was typed
        typed = my_entry.get()
        
        if typed == '':
            data = shows
        else:
            data = []
            for show in shows:
                if typed.lower() in show.lower():
                    data.append(show)
                    
        # Update our listbox with selected items
        update(data)
        # See if the string is within the listbox item list
    my_entry.bind("<KeyRelease>", check)
    
    # Select All Button to show entries as labels
    def transferSelected():
        test = []
        for item in my_listbox.curselection():
            test.append(str.strip((my_listbox.get(item))))
        combined_text = ', '.join(test)
        combined_label = Label(root2, text=combined_text)
        combined_label.grid()
        # Function to transfer the items selected
        store(test, data, root2)
        for item in reversed(my_listbox.curselection()):
            my_listbox.delete(item)

    # Transfer Selected titles
    transferSelected_button = Button(root2, text="Store Selected", command=transferSelected)
    transferSelected_button.grid(pady=10)
    
    
    my_label = Label(root2, text='')
    my_label.grid(pady=5)

#### Progress bars

##### Progress Bars for Storing Anime

##### Progress Bars for Retrieving Anime

#### Rows

##### Row 0

In [15]:
ttk.Label(frm, text="Welcome to the anime library!").grid(column=0, row=0)

##### Row 1

In [16]:
ttk.Label(frm, text="PC anime directory:").grid(column=0, row=1)

if os.path.isdir(data["PC Directory"]):
    ttk.Label(frm, text=data["PC Directory"]).grid(column=1, row=1)

ttk.Button(frm, text="..", command=setPCDir).grid(column=2, row=1)

##### Row 2

In [17]:
ttk.Label(frm, text="Phone anime directory:").grid(column=0, row=2)

if os.path.isdir(data["Phone Directory"]):
    ttk.Label(frm, text=data["Phone Directory"]).grid(column=1, row=2)
    
ttk.Button(frm, text="..", command=setPhoneDir).grid(column=2, row=2)

##### Row 3

In [18]:
# Get the storage usage on pc
if os.path.isdir(data["PC Directory"]):
    total = get_directory_size(data["PC Directory"])
    free = get_available_storage(data["PC Directory"])
    
ttk.Label(frm, text=f"PC storage capacity: {total} / {free / (1024**3):.2f} GB").grid(column=0, row=3)


# TODO:
    # Currently the storage usage is inaccurate
    # How to update the pc usage when the directory is reset

##### Row 4

In [19]:
if os.path.isdir(data["Phone Directory"]):
    total = get_directory_size(data["Phone Directory"])
    free = get_available_storage(data["Phone Directory"])
ttk.Label(frm, text=f"Phone storage capacity: {total} / {free / (1024**3):.2f} GB").grid(column=0, row=4)

##### Row 5

In [20]:
ttk.Button(frm, text="Retrieve", command=transferAnime).grid(column=0, row=5)
ttk.Button(frm, text="Store", command=storeAnime).grid(column=1, row=5)

##### Row 8

In [21]:
root.mainloop()

In [22]:
# COPY THE FILE THEN DELETE IT!!!!!!!!!

# Testing storing information

In [23]:
'''

import json
import math
import os
import shutil
from tkinter import *
from tqdm import tqdm
from tkinter import ttk
from tkinter import filedialog
from tkinter import font
filename = 'information.json'
root3 = Tk()
frm3 = ttk.Frame(root3, padding=10)
frm3.grid()
app_width = 500
app_height = 500
screen_width = root3.winfo_screenwidth()
screen_height = root3.winfo_screenheight()
x = (screen_width / 2) - (app_width / 2)
y = (screen_height / 2) - (app_height / 2)
root3.geometry(f'{app_width}x{app_height}+{int(x)}+{int(y)}')

def test_copy():
    src = "D:\emulatedPC\\avatar"
    dst = "D:\emulatedPhone"
    if os.path.exists(src) and os.path.exists(dst):
        print("Awesome")
    elif not os.path.exists(src):
        ttk.Label(root3, text=f"{src} does not exist!").grid()
    elif not os.path.exists(dst):
        ttk.Label(root3, text=f"{dst} does not exist!").grid()
    

ttk.Button(root3, text="export", command=test_copy).grid()

root3.mainloop()'''

'\n\nimport json\nimport math\nimport os\nimport shutil\nfrom tkinter import *\nfrom tqdm import tqdm\nfrom tkinter import ttk\nfrom tkinter import filedialog\nfrom tkinter import font\nfilename = \'information.json\'\nroot3 = Tk()\nfrm3 = ttk.Frame(root3, padding=10)\nfrm3.grid()\napp_width = 500\napp_height = 500\nscreen_width = root3.winfo_screenwidth()\nscreen_height = root3.winfo_screenheight()\nx = (screen_width / 2) - (app_width / 2)\ny = (screen_height / 2) - (app_height / 2)\nroot3.geometry(f\'{app_width}x{app_height}+{int(x)}+{int(y)}\')\n\ndef test_copy():\n    src = "D:\\emulatedPC\\avatar"\n    dst = "D:\\emulatedPhone"\n    if os.path.exists(src) and os.path.exists(dst):\n        print("Awesome")\n    elif not os.path.exists(src):\n        ttk.Label(root3, text=f"{src} does not exist!").grid()\n    elif not os.path.exists(dst):\n        ttk.Label(root3, text=f"{dst} does not exist!").grid()\n    \n\nttk.Button(root3, text="export", command=test_copy).grid()\n\nroot3.mainl

In [24]:

'''import os
import shutil
import time
import tkinter as tk
from tkinter import ttk
from tqdm import tqdm

# Initialize Tkinter window
root = tk.Tk()
root.title("File Copy Progress")
root.geometry("400x200")

# Progress bar
progress = ttk.Progressbar(root, orient="horizontal", length=300, mode="determinate")
progress.grid(row=4, column=0, pady=20)

# Label to show status
status_label = tk.Label(root, text="Waiting to start...", font=("Arial", 12))
status_label.grid(row=5, column=0,pady=10)

# Function to copy files with tqdm
def copy_files():
    # TODO: Pass src as a series of shows you want to have
    src = r"D:\emulatedPC\avatar"  # Source folder
    
    # TODO: Edit each dst to be a remade destination folder
    dst = r"D:\emulatedPhone\avatar_copy"  # Destination folder

    if not os.path.exists(src):
        status_label.config(text="Source folder does not exist!")
        return

    # Turn 
    files = os.listdir(src)
    total_files = len(files)

    if total_files == 0:
        status_label.config(text="No files to copy!")
        return

    progress["maximum"] = total_files  # Set progress bar max value

    
    # TODO: Add label saying copying {len(files)} files from {src} to {dst}
    
    
    # TODO: Calculate the total storage of all files
    
    
    
    
    # Copy files with tqdm
    for i, file in enumerate(tqdm(files, desc="Copying Files", unit="file")):
        # TODO: Calculate the percentage of files left out of 100
        
        num_items = tk.Label(root, text=f"Copying {len(files)} items from {src} to {dst}", font=("Arial", 12))
        num_items.grid(row=0, column=0,pady=10)
        
        
        # TODO: Add label saying {number} % complete
        percentage_label = tk.Label(root, text=f"{23}% complete", font=("Arial", 12))
        percentage_label.grid(row=1, column=0,pady=10)
        
        
        # TODO: Add label saying name: {file_name}
        name_label = tk.Label(root, text=f"name: {file}", font=("Arial", 12))
        name_label.grid(row=2, column=0,pady=10)
        
        # TODO: Add label saying remaining: _______ (_____ GB)
        remaining_items_label = tk.Label(root, text=f"remaining: {len(files)-i} ({23} KB/MB/GB)", font=("Arial", 12))
        remaining_items_label.grid(row=3, column=0,pady=10)
        
        # TODO: make the dst and src arrays with file being a matrix
        shutil.copytree(os.path.join(src, file), os.path.join(dst, file))
        
        # When I want to remove the tree
        #shutil.rmtree(os.path.join(src, file))
        
        
        
        progress["value"] = i + 1  # Update progress bar
        root.update_idletasks()  # Refresh GUI

    status_label.config(text="Copy Completed!")

# Button to start copying
copy_button = ttk.Button(root, text="Start Copy", command=copy_files)
copy_button.grid(row=6, column=0, pady=20)

# Run the Tkinter main loop
root.mainloop()
'''

'import os\nimport shutil\nimport time\nimport tkinter as tk\nfrom tkinter import ttk\nfrom tqdm import tqdm\n\n# Initialize Tkinter window\nroot = tk.Tk()\nroot.title("File Copy Progress")\nroot.geometry("400x200")\n\n# Progress bar\nprogress = ttk.Progressbar(root, orient="horizontal", length=300, mode="determinate")\nprogress.grid(row=4, column=0, pady=20)\n\n# Label to show status\nstatus_label = tk.Label(root, text="Waiting to start...", font=("Arial", 12))\nstatus_label.grid(row=5, column=0,pady=10)\n\n# Function to copy files with tqdm\ndef copy_files():\n    # TODO: Pass src as a series of shows you want to have\n    src = r"D:\\emulatedPC\x07vatar"  # Source folder\n    \n    # TODO: Edit each dst to be a remade destination folder\n    dst = r"D:\\emulatedPhone\x07vatar_copy"  # Destination folder\n\n    if not os.path.exists(src):\n        status_label.config(text="Source folder does not exist!")\n        return\n\n    # Turn \n    files = os.listdir(src)\n    total_files = l